# 신경망 하이퍼파라미터 튜닝 방법
`하이퍼파라미터`란, 개발자가 직접 조절해주는 변수로, 모델 성능에 큰 영향을 미친다.
+ 배치 크기
+ 에포크(epochs)
+ 옵티마이저(optimizers)
+ 학습률(learning rate)
+ 활성화 함수(activation)
+ Regularization(weight decay, Dropout 등)
+ 은닉층(Hidden layer)의 노드(Node) 수
+ etc

In [1]:
# 구글 드라이브 마운트(cjyjob1993@gmail.com)
from google.colab import drive
drive.mount('/content/drive')

# lib 디렉토리를을 환경 변수에 추가
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/myCode/lib')

Mounted at /content/drive


In [2]:
# 커스텀 라이브러리 임포트
from func_debug_log import debug

In [3]:
# debug_log (0 : 미출력, 1 : 함수 실행 로그 출력, 2 : 함수 내부)
debug_flag = 1

## 1. 교차 검증 적용하기

In [4]:
if debug_flag in range(1, 3) : debug('신경망에 교차 검증 적용하기', __name__)

2022.12.02 19:30:22 __main__ 신경망에 교차 검증 적용하기


In [5]:
if debug_flag in range(1, 3) : debug('데이터 불러오기', __name__)

from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

2022.12.02 19:30:22 __main__ 데이터 불러오기
57026/57026 [==============================] - 0s 0us/step


In [6]:
if debug_flag in range(1, 3) : debug('라이브러리 임포트', __name__)

from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import pandas as pd
import tensorflow as tf
import os

2022.12.02 19:30:22 __main__ 라이브러리 임포트


In [7]:
if debug_flag in range(1, 3) : debug('KFold 설정', __name__)

kf = KFold(n_splits = 5)
skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True) 

2022.12.02 19:30:22 __main__ KFold 설정


In [8]:
if debug_flag in range(1, 3) : debug('신경망 모델 구축', __name__)

model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

2022.12.02 19:30:22 __main__ 신경망 모델 구축


In [9]:
if debug_flag in range(1, 3) : debug('교차 검증 적용', __name__)

x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)


for train_index, val_index in kf.split(np.zeros(x_train.shape[0])):
  if debug_flag in range(2, 3) : debug('데이터 Fold 사용', __name__)
  training_data = x_train.iloc[train_index, :]
  training_data_label = y_train.iloc[train_index]
  validation_data = x_train.iloc[val_index, :]
  validation_data_label = y_train.iloc[val_index]

  if debug_flag in range(2, 3) : debug('신경망 모델 컴파일', __name__)
  model.compile(loss='mean_squared_error', optimizer='adam')

  if debug_flag in range(2, 3) : debug('신경망 모델 교차 검증 적용 학습', __name__)
  model.fit(x_train, y_train,
            epochs=10,
            batch_size=32,
            validation_data = (validation_data, validation_data_label),
            )
  
  if debug_flag in range(2, 3) : debug('신경망 모델 평가', __name__)
  results = model.evaluate(x_test, y_test, batch_size=32)
  print("test loss, test mse:", results)

2022.12.02 19:30:22 __main__ 교차 검증 적용
Epoch 1/10
13/13 [==============================] - 1s 17ms/step - loss: 2490.7585 - val_loss: 1158.0894
Epoch 2/10
13/13 [==============================] - 0s 4ms/step - loss: 428.1286 - val_loss: 234.4010
Epoch 3/10
13/13 [==============================] - 0s 4ms/step - loss: 167.4667 - val_loss: 136.2015
Epoch 4/10
13/13 [==============================] - 0s 5ms/step - loss: 99.2333 - val_loss: 52.5970
Epoch 5/10
13/13 [==============================] - 0s 5ms/step - loss: 69.2980 - val_loss: 57.0353
Epoch 6/10
13/13 [==============================] - 0s 4ms/step - loss: 64.7647 - val_loss: 47.9996
Epoch 7/10
13/13 [==============================] - 0s 5ms/step - loss: 60.1913 - val_loss: 51.9230
Epoch 8/10
13/13 [==============================] - 0s 4ms/step - loss: 60.2250 - val_loss: 46.7221
Epoch 9/10
13/13 [==============================] - 0s 5ms/step - loss: 58.8752 - val_loss: 45.8655
Epoch 10/10
4/4 [==============================] - 0s

## 2. 하이퍼파라미터 튜닝 방식

### [1] Babysitting (육아) or Grad Student Descent (대학원생 갈아넣기)
+ 수작업으로 하이퍼파라미터 조정

### [2] Grid Search (격자 탐색)
+ 하이퍼파라미터마다 탐색 지점을 규칙적으로 지정하여, 모든 지점의 조합을 수행한다(파라미터의 중요도가 모두 동일하다)
+ 조합의 수가 많아, 소요 시간이 매우 길다
+ 주요 하이퍼파라미터 최대 2개 정도의 최적값을 찾는 용도에 적합하다

### [3] Random Search (무작위 탐색)
+ 지정된 범위 내에서 무작위로 하이퍼파라미터를 조합하여 수행한 뒤 최고 성능의 모델을 반환한다(최적의 조합을 보장하지 않는다)
+ 시도 횟수를 지정하여, 소요 시간을 조절할 수 있다
+ 탐색과정에서 상대적으로 중요한 파라미터에 대해 집중하여 탐색하도록 최적화되어있다

### [4] Bayesian Methods (베이지안 탐색)
+ 이전 탐색 결과 정보를 새로운 탐색에 활용하는 방법.
+ `bayes_opt`나 `hyperopt`와 같은 패키지로 적용이 가능

In [10]:
if debug_flag in range(1, 3) : debug('Grid Search 예시', __name__)

2022.12.02 19:30:22 __main__ Grid Search 예시


In [11]:
if debug_flag in range(1, 3) : debug('라이브러리 임포트', __name__)
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

2022.12.02 19:30:22 __main__ 라이브러리 임포트


In [12]:
if debug_flag in range(2, 3) : debug('numpy seed 고정', __name__)
numpy.random.seed(42)

In [13]:
if debug_flag in range(1, 3) : debug('데이터 불러오기', __name__)
url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

dataset = pd.read_csv(url, header=None).values

2022.12.02 19:30:22 __main__ 데이터 불러오기


In [14]:
if debug_flag in range(2, 3) : debug('데이터 분할', __name__)
X = dataset[:,0:8]
Y = dataset[:,8]

In [15]:
if debug_flag in range(1, 3) : debug('신경망 모델 제작', __name__)
def create_model():
    model = Sequential()
    model.add(Dense(100, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

2022.12.02 19:30:22 __main__ 신경망 모델 제작


In [16]:
if debug_flag in range(2, 3) : debug('KerasClassifier로 wrapping', __name__)
model = KerasClassifier(build_fn=create_model, verbose=0)

<ipython-input-16-7c1d537425fb>:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [17]:
if debug_flag in range(2, 3) : debug('하이퍼파라미터 탐색 범위 설정', __name__)
batch_size = [8, 16, 32, 64, 128]
param_grid = dict(batch_size=batch_size)

In [18]:
if debug_flag in range(1, 3) : debug('GridSearchCV 지정 학습', __name__)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

2022.12.02 19:30:22 __main__ GridSearchCV 지정 학습


In [19]:
if debug_flag in range(1, 3) : debug('최적의 하이퍼파라미터와 결과 출력', __name__)
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

2022.12.02 19:30:22 __main__ 최적의 하이퍼파라미터와 결과 출력
Best: 0.6145828008651734 using {'batch_size': 8}
Means: 0.6145828008651734, Stdev: 0.030726731512724127 with: {'batch_size': 8}
Means: 0.6067736268043518, Stdev: 0.06031485296328812 with: {'batch_size': 16}
Means: 0.5548510372638702, Stdev: 0.09645475559668859 with: {'batch_size': 32}
Means: 0.5103726327419281, Stdev: 0.08884159609645798 with: {'batch_size': 64}
Means: 0.41247772574424746, Stdev: 0.13615447657903806 with: {'batch_size': 128}


## 3. 라이브러리(Keras Tuner)를 사용한 하이퍼 파라미터 튜닝
`Keras Tuner` 는 케라스 프레임워크에서 하이퍼파라미터를 튜닝하는 데 도움이 되는 라이브러리입니다.

# Fashion MNIST 예제

In [20]:
if debug_flag in range(1, 3) : debug('keras Tuner 예시', __name__)

2022.12.02 19:30:22 __main__ keras Tuner 예시


In [21]:
if debug_flag in range(2, 3) : debug('keras Tuner 설치', __name__)
!pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 47.0 MB/s 


In [22]:
if debug_flag in range(1, 3) : debug('라이브러리 임포트', __name__)
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten

import tensorflow as tf
import IPython

import kerastuner as kt

2022.12.02 19:30:22 __main__ 라이브러리 임포트


<ipython-input-22-076806c1e529>:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [23]:
if debug_flag in range(1, 3) : debug('데이터 불러오기', __name__)
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

2022.12.02 19:30:22 __main__ 데이터 불러오기
4422102/4422102 [==============================] - 0s 0us/step


In [24]:
if debug_flag in range(2, 3) : debug('이미지 정규화', __name__)
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [25]:
if debug_flag in range(1, 3) : debug('신경망 모델 제작', __name__)
def model_builder(hp):
  model = keras.Sequential()
  model.add(Flatten(input_shape=(28, 28)))
  
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(Dense(units = hp_units, activation = 'relu'))
  model.add(Dense(10, activation='softmax'))

  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

2022.12.02 19:30:22 __main__ 신경망 모델 제작


In [26]:
if debug_flag in range(1, 3) : debug('keras Tuner 지정', __name__)
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

2022.12.02 19:30:22 __main__ keras Tuner 지정


In [27]:
if debug_flag in range(1, 3) : debug('Callback 함수 지정', __name__)
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

2022.12.02 19:30:22 __main__ Callback 함수 지정


In [28]:
if debug_flag in range(1, 3) : debug('하이퍼파라미터 탐색', __name__)
tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 {best_hps.get('units')} 입니다.
최적의 학습 속도는 {best_hps.get('learning_rate')} 입니다.
""")

Trial 30 Complete [00h 02m 23s]
val_accuracy: 0.8737000226974487

Best val_accuracy So Far: 0.8889999985694885
Total elapsed time: 00h 26m 53s

하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 224 입니다.
최적의 학습 속도는 0.001 입니다.



In [30]:
if debug_flag in range(2, 3) : debug('최적 파라미터로 재학습', __name__)
model = tuner.hypermodel.build(best_hps)
model.summary()
model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 224)               175840    
                                                                 
 dense_5 (Dense)             (None, 10)                2250      
                                                                 
Total params: 178,090
Trainable params: 178,090
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.4817 - accuracy: 0.8282 - val_loss: 0.4307 - val_accuracy: 0.8464
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3635 - accuracy: 0.8667 - val_loss: 0.3671 - val_accuracy: 0.8635
Epoch 3/10
1875/1875 [==